In [ ]:
from xmlrpc.server import SimpleXMLRPCServer
import xml.etree.ElementTree as ET
import os
from datetime import datetime
from threading import Thread


class ThreadedXMLRPCServer(SimpleXMLRPCServer):
    def process_request_thread(self, request, client_address):
        """Handle a request in a new thread."""
        try:
            self.finish_request(request, client_address)
            self.shutdown_request(request)
        except Exception as e:
            self.handle_error(request, client_address)
            self.shutdown_request(request)
    
    def process_request(self, request, client_address):
        """Start a new thread to process the request."""
        thread = Thread(target=self.process_request_thread, args=(request, client_address))
        thread.daemon = True
        thread.start()

xml_file = 'db.xml'
if not os.path.exists(xml_file):
    root = ET.Element("data")
    tree = ET.ElementTree(root)
    tree.write(xml_file)
else:
    tree = ET.parse(xml_file)
    root = tree.getroot()

def add_note(topic, text, timestamp=None):
    if timestamp is None:
        timestamp = datetime.now().strftime("%m/%d/%y - %H:%M:%S")
    found = False
    for t in root.findall('topic'):
        if t.get('name') == topic:
            found = True
            break
    if not found:
        t = ET.SubElement(root, 'topic', name=topic)
    note = ET.SubElement(t, 'note', name=f"Note {timestamp}")
    ET.SubElement(note, 'text').text = text
    ET.SubElement(note, 'timestamp').text = timestamp
    tree.write(xml_file)
    return True

def get_notes(topic):
    notes = []
    for t in root.findall('topic'):
        if t.get('name') == topic:
            for note in t.findall('note'):
                note_text = note.find('text').text
                notes.append(note_text)
            break
    return notes
import requests

def query_wikipedia_summary(topic):
    base_url = "https://en.wikipedia.org/api/rest_v1/page/summary/"
    try:
        response = requests.get(base_url + topic)
        if response.status_code == 200:
            data = response.json()
            summary = data.get('extract', 'Summary not available.')
            return summary
        return "Failed to retrieve summary from Wikipedia."
    except requests.RequestException:
        return "Request to Wikipedia failed."


def add_wikipedia_info(topic):
    wiki_summary = query_wikipedia_summary(topic)
    if not wiki_summary.startswith("Failed"):
        for t in root.findall('topic'):
            if t.get('name') == topic:
                note = ET.SubElement(t, 'note', name="Wikipedia Summary")
                ET.SubElement(note, 'text').text = wiki_summary
                tree.write(xml_file)
                return f"Added Wikipedia summary to topic '{topic}'."
        return "Topic not found in database."
    return wiki_summary

with ThreadedXMLRPCServer(('localhost', 8000)) as server:
    server.register_function(add_note, "add_note")
    server.register_function(get_notes, "get_notes")
    server.register_function(add_wikipedia_info, "add_wikipedia_info")
    print("Server is running...")
    server.serve_forever()


Server is running...


127.0.0.1 - - [10/Mar/2024 17:13:55] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2024 17:14:05] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2024 17:14:27] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2024 17:14:34] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [10/Mar/2024 17:14:51] "POST / HTTP/1.1" 200 -
